In [4]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from typing import List
driver = webdriver.Chrome('./chromedriver.exe')

# 페이지 크롤링 함수
---

In [27]:


def crawling():
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')

    content = soup.select('table.table_develop')
    ex = content[0].text.replace('\n', " ")
    ex = ex.split(' ')
    ex = ex[25:]
    date = []
    for s in ex:
        if s != '':
            date.append(s)
    return date

In [ ]:
# 12달이므로 12칸의 리스트 생성
def make_list():
    List = []
    for i in range(13):
        List.append([])
    return List

# 평균온도
---

In [28]:

def get_avg_temp(year: str)->List:
    driver.get(
        'https://www.weather.go.kr/weather/climate/past_table.jsp?stn=159&yy={}&obs=07&x=9&y=18'
        .format(year))
    avg_temp = make_list()
    date = crawling()
    cnt = 0
    for num in date:
        avg_temp[cnt % 13].append(num)
        cnt += 1

    avg_temp = avg_temp[1:]

    avg_temp = pd.DataFrame(avg_temp).T
    avg_temp.drop(31, axis=0, inplace=True)
    avg_temp_final = []
    for i in range(12):
        for j in range(31):
            avg_temp_final.append(avg_temp.iloc[j, i])
    return avg_temp_final

# 평균 강수량 
---

In [33]:
def get_avg_rain(year: str) -> List:
    driver.get(
        'https://www.weather.go.kr/weather/climate/past_table.jsp?stn=159&yy={}&obs=21&x=19&y=4'
        .format(year))
    date = crawling()
    avg_rain = make_list()
    cnt = 0
    for num in date:

        avg_rain[cnt % 13].append(num)
        cnt += 1

    avg_rain = avg_rain[1:]

    avg_rain = pd.DataFrame(avg_rain).T
    avg_rain.drop(31, axis=0, inplace=True)
    avg_rain_final = []

    for i in range(12):
        for j in range(31):
            if avg_rain.iloc[j][i] == '\xa0':
                avg_rain_final.append(0)
            else:
                avg_rain_final.append(avg_rain.iloc[j, i])
    return avg_rain_final

# 일조시간
---

In [38]:
def get_avg_sunny(year: str) -> List:
    driver.get(
        'https://www.weather.go.kr/weather/climate/past_table.jsp?stn=159&yy={}&obs=35&x=18&y=6'
        .format(year))
    date = crawling()

    avg_sunny = make_list()
    cnt = 0
    for num in date:

        avg_sunny[cnt % 13].append(num)
        cnt += 1

    avg_sunny = avg_sunny[1:]

    avg_sunny = pd.DataFrame(avg_sunny).T
    avg_sunny.drop(31, axis=0, inplace=True)
    avg_sunny_final = []

    for i in range(12):
        for j in range(31):
            avg_sunny_final.append(avg_sunny.iloc[j, i])
    return avg_sunny_final

# 평균 운량
---

In [40]:
def get_avg_cloud(year: int) -> List:
    driver.get(
        'https://www.weather.go.kr/weather/climate/past_table.jsp?stn=159&yy={}&obs=59&x=24&y=8'
        .format(year))

    date = crawling()
    avg_cloud = make_list()
    cnt = 0
    for num in date:

        avg_cloud[cnt % 13].append(num)
        cnt += 1

    avg_cloud = avg_cloud[1:]

    avg_cloud = pd.DataFrame(avg_cloud).T
    avg_cloud.drop(31, axis=0, inplace=True)
    avg_cloud_final = []

    for i in range(12):
        for j in range(31):
            avg_cloud_final.append(avg_cloud.iloc[j, i])
    return avg_cloud_final

# 각년도별 데이터 수집후 결합
--- 
2015-2019 년도의 데이터를 각각 수집한후 하나의 df 로 결합시킨다.

In [72]:
year = ['2015','2016','2017','2018','2019']
columns = ['평균기온','강수량','일조시간','운량']
df=pd.DataFrame()
for y in year:
    globals()['df_{}'.format(y)] = [get_avg_temp(y),get_avg_rain(y),get_avg_sunny(y),get_avg_cloud(y)]
    for _ in globals()['df_{}'.format(y)]:
        # 데이터 중간에 이상치가 있어서 제거해주는작업.
        for i in [340,278,185,123,61,60,59]:
            del _[i]
    globals()['df_{}'.format(y)] = pd.DataFrame(globals()['df_{}'.format(y)]).T
    globals()['df_{}'.format(y)].columns = columns

In [75]:
#5개 데이터 프레임을 합쳐준다.

df = df_2015.append(df_2016,ignore_index=True)
df = df.append(df_2017,ignore_index=True)
df = df.append(df_2018,ignore_index=True)
df = df.append(df_2019,ignore_index=True)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   평균기온    1825 non-null   object
 1   강수량     1825 non-null   object
 2   일조시간    1825 non-null   object
 3   운량      1825 non-null   object
dtypes: object(4)
memory usage: 57.2+ KB
None


,평균기온,강수량,일조시간,운량
0,-2.2,0,9.2,0.4
1,-1.0,0,9.2,0.1
2,1.8,0,9.4,2.8
3,7.0,0,8.8,1.1
4,9.3,0.5,5.1,6.5


In [76]:
# 데이터 프레임의 인덱스를 연-월-일 로바꿔준다.
date = pd.DataFrame({'date':pd.date_range('2015-01-01', periods=365*5, freq='1D')})
date= date['date'].astype(str).tolist()
df['날짜']=date
df.columns=['평균기온','강수량','일조시간','운량','날짜']
df=df[['날짜','평균기온','강수량','일조시간','운량']]
df.head()

,날짜,평균기온,강수량,일조시간,운량
0,2015-01-01,-2.2,0,9.2,0.4
1,2015-01-02,-1.0,0,9.2,0.1
2,2015-01-03,1.8,0,9.4,2.8
3,2015-01-04,7.0,0,8.8,1.1
4,2015-01-05,9.3,0.5,5.1,6.5


# 배추 가격 데이터와 날씨 데이터 결합 및 저장
---

In [79]:
# 저장된 배추 가겨 데이터를 불러온다.
price= pd.read_csv('배추가격데이터.csv',index_col=0)
price.head()

,날짜,배추가격평균
1,2015-01-02,"4,000"
2,2015-01-05,"4,000"
3,2015-01-06,"4,000"
4,2015-01-07,"4,000"
5,2015-01-08,"4,000"


In [81]:
# 배추가격은 평일 데이터만 존재하므로 배추가격데이터를 기준으로 날짜데이터와 결합한다.
_=pd.merge(left=df,
               right=배추,
               how='inner',
               left_on='날짜',
               right_on='날짜')

final_df= _
print(final_df.info())
final_df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226 entries, 0 to 1225
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      1226 non-null   object
 1   평균기온    1226 non-null   object
 2   강수량     1226 non-null   object
 3   일조시간    1226 non-null   object
 4   운량      1226 non-null   object
 5   배추가격평균  1226 non-null   object
dtypes: object(6)
memory usage: 67.0+ KB
None


,날짜,평균기온,강수량,일조시간,운량,배추가격평균
0,2015-01-02,-1.0,0,9.2,0.1,"4,000"
1,2015-01-05,9.3,0.5,5.1,6.5,"4,000"
2,2015-01-06,7.6,26.0,8.6,3.1,"4,000"
3,2015-01-07,1.4,0,9.1,2.5,"4,000"
4,2015-01-08,0.8,0,9.4,0.0,"4,000"


In [ ]:
# 크롤링 및 결합 이 끝난 데이터를 저장한다.
final_df.to_csv('final_df.csv',index=False)